In [ ]:
!pip install musdb

In [1]:
import librosa
import math
import numpy as np
import os
import pickle
import sys
import scipy.io.wavfile

import musdb
mus = musdb.DB(root_dir='/mnt/z/data/musdb18/raw')

In [2]:
SAMPLE_RATE = 44100
TARGET_SAMPLE_RATE = 16000
tracks = mus.load_mus_tracks()

idx = np.array(range(len(tracks)))
np.random.seed(12345678)
np.random.shuffle(idx)
num_train = int(math.ceil(0.8 * len(tracks)))

output_dir = '/mnt/z/data/musdb18'
os.mkdir(os.path.join(output_dir, 'train'))
os.mkdir(os.path.join(output_dir, 'test'))

for i, track in enumerate(tracks):
    vocal = track.targets['vocals'].audio
    off_vocal = track.targets['accompaniment'].audio
    # Stereo to mono and resample.
    vocal = (vocal[:, 0] + vocal[:, 1]) / 2
    vocal = librosa.core.resample(vocal, SAMPLE_RATE, TARGET_SAMPLE_RATE)
    vocal = (vocal * 32767).astype(np.int16)
    off_vocal = (off_vocal[:, 0] + off_vocal[:, 1]) / 2
    off_vocal = librosa.core.resample(off_vocal, SAMPLE_RATE, TARGET_SAMPLE_RATE)
    off_vocal = (off_vocal * 32767).astype(np.int16)
    
    data = np.stack((off_vocal, vocal), -1)
    output = os.path.join(output_dir, 
                          'test' if i >= num_train else 'train', 
                          track.name + '.wav')
    print(output)
    scipy.io.wavfile.write(output, TARGET_SAMPLE_RATE, data)
    
print('Done')

/mnt/z/data/musdb18/train/A Classic Education - NightOwl.wav
/mnt/z/data/musdb18/train/ANiMAL - Clinic A.wav
/mnt/z/data/musdb18/train/ANiMAL - Easy Tiger.wav
/mnt/z/data/musdb18/train/ANiMAL - Rockshow.wav
/mnt/z/data/musdb18/train/Actions - Devil's Words.wav
/mnt/z/data/musdb18/train/Actions - One Minute Smile.wav
/mnt/z/data/musdb18/train/Actions - South Of The Water.wav
/mnt/z/data/musdb18/train/Aimee Norwich - Child.wav
/mnt/z/data/musdb18/train/Alexander Ross - Goodbye Bolero.wav
/mnt/z/data/musdb18/train/Alexander Ross - Velvet Curtain.wav
/mnt/z/data/musdb18/train/Angela Thomas Wade - Milk Cow Blues.wav
/mnt/z/data/musdb18/train/Atlantis Bound - It Was My Fault For Waiting.wav
/mnt/z/data/musdb18/train/Auctioneer - Our Future Faces.wav
/mnt/z/data/musdb18/train/AvaLuna - Waterduct.wav
/mnt/z/data/musdb18/train/BigTroubles - Phantom.wav
/mnt/z/data/musdb18/train/Bill Chudziak - Children Of No-one.wav
/mnt/z/data/musdb18/train/Black Bloc - If You Want Success.wav
/mnt/z/data/musd

/mnt/z/data/musdb18/test/The Sunshine Garcia Band - For I Am The Moon.wav
/mnt/z/data/musdb18/test/Timboz - Pony.wav
/mnt/z/data/musdb18/test/Tom McKenzie - Directions.wav
/mnt/z/data/musdb18/test/Triviul feat. The Fiend - Widow.wav
/mnt/z/data/musdb18/test/We Fell From The Sky - Not You.wav
/mnt/z/data/musdb18/test/Zeno - Signs.wav
Done
